# Pulling annotations from KBase genome

In [5]:
%run util.py
gen_ref = "219173/Acinetobacter_baylyi_ADP1_RPG_Snekmer_LA_Pfam"
annoapi = util.anno_client(native_python_api=False)
annoont = AnnotationOntology.from_kbase_data(annoapi.get_annotation_ontology_events({
    "input_ref" : gen_ref
}),gen_ref,util.module_dir+"/data/")
output = {}
for gene in annoont.genes:
    output[gene] = {}
    for event in annoont.genes[gene].event_terms:
        output[gene][event] = {}
        for term in annoont.genes[gene].event_terms[event]:
            output[gene][event][term] = 1
util.save("gene_term_hash", output)

Output files printed to:/Users/chenry/Dropbox/Contexts/Data/ADP1/nboutput when using KBDevUtils.output_dir


1754025249.821354 INFO: get_annotation_ontology_events:{
    "input_ref": "219173/Acinetobacter_baylyi_ADP1_RPG_Snekmer_LA_Pfam"
}


Loading ModelSEEDBiochem from /Users/chenry/code/ModelSEEDDatabase


# Pull the ModelSEED model

In [ ]:
%run util.py
output = util.get_model_and_simulate("179225/Abaylyi_ADP1_RASTMS2_OMEGGA_Abaylyi_ADP1_RAST.mdlMS2_OMEGGA_iAbaylyi_Carbon_Succinic.gf", "KBaseMedia/Carbon-Pyruvic-Acid")
annohash = util.load("gene_term_hash")
annohash = util.add_model_data_to_annotations(output["model"],annohash,output["rxn_flux_data"],"modelseed")
util.save("gene_term_hash", annohash)

Output files printed to:/Users/chenry/Dropbox/Contexts/Data/ADP1/nboutput when using KBDevUtils.output_dir
Loading ModelSEEDBiochem from /Users/chenry/code/ModelSEEDDatabase


1754025255.1922739 INFO: metabolites 1270
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x10732ad00>>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/modelseed/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


# Pull the published model

In [ ]:
%run util.py
output = util.get_model_and_simulate("179225/iAbaylyi_ADP1", "KBaseMedia/Carbon-Pyruvic-Acid")
annohash = util.load("gene_term_hash")
annohash = util.add_model_data_to_annotations(output["model"],annohash,output["rxn_flux_data"],"published")
util.save("gene_term_hash", annohash)

Output files printed to:/Users/chenry/Dropbox/Contexts/Data/ADP1/nboutput when using KBDevUtils.output_dir
Loading ModelSEEDBiochem from /Users/chenry/code/ModelSEEDDatabase


1749676178.786598 INFO: metabolites 828
1749676200.8465338 INFO: reactions 874
1749676200.870972 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1749676200.9477608 INFO: Default biomass: [bio1]
1749676201.0879319 INFO: cpd00048 not found in model!
1749676201.088316 INFO: Media compound: cpd00048 not found in model.
1749676201.088556 INFO: cpd00063 not found in model!
1749676201.088784 INFO: Media compound: cpd00063 not found in model.
1749676201.089018 INFO: cpd10516 not found in model!
1749676201.0892649 INFO: Media compound: cpd10516 not found in model.
1749676201.0897212 INFO: cpd00205 not found in model!
1749676201.090019 INFO: Media compound: cpd00205 not found in model.
1749676201.0903559 INFO: cpd00254 not found in model!
1749676201.090658 INFO: Media compound: cpd00254 not found in model.
1749676201.09102 INFO: cpd00099 not found in model!
1749676201.0914571 INFO: Media compound: cpd00099 not found in model.
1749676201.0917811 INFO: cpd

# Adding proteomics data to the spreadsheet

In [10]:
%run util.py
gene_term_hash = util.load("gene_term_hash")

# Load Tukey tab from Excel
#tukey_df = pd.read_excel("data/ADP1_Proteomics_July2024_anova.xlsx", sheet_name="Tukey")
tukey_df = pd.read_excel("data/UGA_Proteomics_May2025_Report.xlsx", sheet_name="Tukey", skiprows=1)

# Iterate over each row and add to gene_term_hash
for idx, row in tukey_df.iterrows():
    gene = row['variable']
    group1 = row['group1']
    group2 = row['group2']
    estimate = row['estimate']
    p_adj = row['p.adj']
    key = f"{group1}_vs_{group2}"
    if gene not in gene_term_hash:
        gene_term_hash[gene] = {}
    if p_adj < 0.05:
        gene_term_hash[gene][key] = {"value":str(estimate)}
    else:
        gene_term_hash[gene][key] = {"value":"0"}
# Optionally save updated gene_term_hash
util.save("gene_term_hash", gene_term_hash)

Output files printed to:/Users/chenry/Dropbox/Contexts/Data/ADP1/nboutput when using KBDevUtils.output_dir
Loading ModelSEEDBiochem from /Users/chenry/code/ModelSEEDDatabase


# Load all dictionaries 

In [ ]:
%run util.py
ec_dict = util.load("EC_dictionary")
ko_dict = util.load("KO_dictionary")
pf_dict = util.load("PF_dictionary")
sso_dict = util.load("SSO_dictionary")

all_terms = {}
for d in [ec_dict, ko_dict, pf_dict, sso_dict]:
    if d and "term_hash" in d:
        for term_id, term in d["term_hash"].items():
            all_terms[term_id] = term["name"]
util.save("all_terms", all_terms)

Output files printed to:/Users/chenry/Dropbox/Contexts/Data/ADP1/nboutput when using KBDevUtils.output_dir
Loading ModelSEEDBiochem from /Users/chenry/code/ModelSEEDDatabase


# Adding names to gene term hash

In [11]:
%run util.py
all_terms = util.load("all_terms")
gene_term_hash = util.load("gene_term_hash")

def replace_term_ids_with_names(gene_term_hash, all_terms):
    for gene, events in gene_term_hash.items():
        for event, terms in events.items():
            for term_id in list(terms.keys()):
                if term_id in all_terms:
                    terms[term_id] = all_terms[term_id]
    return gene_term_hash

gene_term_hash_named = replace_term_ids_with_names(gene_term_hash, all_terms)
util.save("gene_term_hash_named", gene_term_hash_named)

Output files printed to:/Users/chenry/Dropbox/Contexts/Data/ADP1/nboutput when using KBDevUtils.output_dir
Loading ModelSEEDBiochem from /Users/chenry/code/ModelSEEDDatabase


# Translating annotations into a spreadsheet

In [14]:
%run util.py
import pandas as pd

# Load the named gene-term hash
gene_term_hash_named = util.load("gene_term_hash_named")

# Define mapping from annotation source substrings to column names
source_map = {
    "RAST-annotate_genome:1.9.1:SSO:2025-06-06T03:38:24": "RAST",
    "GLM4ECModule.annotate_microbes_with_GLM4EC:0.1.1.am:EC:2025-06-06 03:51:01": "GLM4EC",
    "DRAM:0.1.2:KO:2025_06_06_04_17_25": "DRAM",
    "Snekmer Apply:1.0:PF:2025.06.06-04:57:01AM": "Sneckmer",
    "modelstatus": "Model Status",
    "reactions": "Reactions",
    "modelseed_flux": "MS flux",
    "published_flux": "Published flux",
    "ACN2586_proteome": "ACN2586 proteome",
    "ACN2821_proteome": "ACN2821 proteome",
    "cluster": "Cluster",
    "cluster_size": "Cluster Size",
    "ACN2586_vs_ACN2821": "ACN2586 vs ACN2821",
    # "ACN2586_vs_ACN3015": "ACN2586 vs ACN3015",
    # "ACN2586_vs_ACN3468": "ACN2586 vs ACN3468",
    # "ACN2586_vs_ACN3471": "ACN2586 vs ACN3471",
    # "ACN2586_vs_ACN3474": "ACN2586 vs ACN3474",
    # "ACN2586_vs_ACN3477": "ACN2586 vs ACN3477",
    # "ACN2586_vs_ADP1": "ACN2586 vs ADP1",
    # "ACN2821_vs_ACN3015": "ACN2821 vs ACN3015",
    # "ACN2821_vs_ACN3468": "ACN2821 vs ACN3468",
    # "ACN2821_vs_ACN3471": "ACN2821 vs ACN3471",
    # "ACN2821_vs_ACN3474": "ACN2821 vs ACN3474",
    # "ACN2821_vs_ACN3477": "ACN2821 vs ACN3477",
    # "ACN2821_vs_ADP1": "ACN2821 vs ADP1",
    # "ACN3015_vs_ACN3468": "ACN3015 vs ACN3468",
    # "ACN3015_vs_ACN3471": "ACN3015 vs ACN3471",
    # "ACN3015_vs_ACN3474": "ACN3015 vs ACN3474",
    # "ACN3015_vs_ACN3477": "ACN3015 vs ACN3477",
    # "ACN3015_vs_ADP1": "ACN3015 vs ADP1",
    # "ACN3468_vs_ACN3471": "ACN3468 vs ACN3471",
    # "ACN3468_vs_ACN3474": "ACN3468 vs ACN3474",
    # "ACN3468_vs_ACN3477": "ACN3468 vs ACN3477",
    # "ACN3468_vs_ADP1": "ACN3468 vs ADP1",
    # "ACN3471_vs_ACN3474": "ACN3471 vs ACN3474",
    # "ACN3471_vs_ACN3477": "ACN3471 vs ACN3477",
    # "ACN3471_vs_ADP1": "ACN3471 vs ADP1",
    # "ACN3474_vs_ACN3477": "ACN3474 vs ACN3477",
    # "ACN3474_vs_ADP1": "ACN3474 vs ADP1",
    # "ACN3477_vs_ADP1": "ACN3477 vs ADP1"
}

# Prepare data for DataFrame
rows = []
for gene, sources in gene_term_hash_named.items():
    row = {"Gene": gene}
    for src_key, terms in sources.items():
        if isinstance(terms, dict):
            for src_sub, col in source_map.items():
                if src_sub == src_key:
                    # Join all annotation names for this source
                    val = "; ".join(str(v).strip() for v in terms.values())
                    row[col] = val
        else:
            # Handle cases where the source is not in the mapping
            row[src_key] = str(terms).strip()
    rows.append(row)

df = pd.DataFrame(rows)
#df = df[["Gene", "RAST", "GLM4EC", "DRAM", "Sneckmer", "ModelSEED", "Published", "Flux","ACN2586 vs ACN2821","ACN2586 vs ACN3015","ACN2586 vs ACN3468","ACN2586 vs ACN3471","ACN2586 vs ACN3474","ACN2586 vs ACN3477","ACN2586 vs ADP1","ACN2821 vs ACN3015","ACN2821 vs ACN3468","ACN2821 vs ACN3471","ACN2821 vs ACN3474","ACN2821 vs ACN3477","ACN2821 vs ADP1","ACN3015 vs ACN3468","ACN3015 vs ACN3471","ACN3015 vs ACN3474","ACN3015 vs ACN3477","ACN3015 vs ADP1","ACN3468 vs ACN3471","ACN3468 vs ACN3474","ACN3468 vs ACN3477","ACN3468 vs ADP1","ACN3471 vs ACN3474","ACN3471 vs ACN3477","ACN3471 vs ADP1","ACN3474 vs ACN3477","ACN3474 vs ADP1","ACN3477 vs ADP1"]]
# Save to Excel
df.to_excel("gene_annotations.xlsx", index=False)

2025-09-26 10:26:49,672 - __main__.NotebookUtil - INFO - Loaded 0 tokens from /Users/chenry/.tokens
2025-09-26 10:26:49,672 - __main__.NotebookUtil - INFO - Loaded kbase tokens from /Users/chenry/.kbase/token
2025-09-26 10:26:49,673 - __main__.NotebookUtil - WARNING - Section 'ModelSEEDBiochem' not found in config
2025-09-26 10:26:49,673 - __main__.NotebookUtil - INFO - ModelSEED database loaded from /Users/chenry/Dropbox/SyncCode/KBUtilLib/src/kbutillib/dependencies/ModelSEEDDatabase


/Users/chenry/Dropbox/SyncCode/KBUtilLib/src


2025-09-26 10:26:50,033 - __main__.NotebookUtil - INFO - Notebook environment detected
2025-09-26 10:26:50,033 - __main__.NotebookUtil - INFO - ArgoGatewayClient initialised | model=gpto3mini env=dev timeout=120.0s url=https://apps-dev.inside.anl.gov/argoapi/api/v1/resource/streamchat/


# Getting genome object from KBase

In [ ]:
%run util.py
ws = 219173
id = "Acinetobacter_baylyi_ADP1_RPG_Snekmer_LA_Pfam"
data = util.get_object(id,ws)
util.save("ADP1Genome",data)

python version 3.9.19
KBBaseModules 0.0.1
modelseedpy 0.4.2
cobrakbase 0.4.0
Output files printed to:/Users/chenry/Dropbox/Contexts/Data/ADP1/nboutput when using KBDevUtils.output_dir
Loading ModelSEEDBiochem from /Users/chenry/code/ModelSEEDDatabase


# Translating Gene Calls in Spreadsheet to GFF

In [ ]:
%run util.py

def convert_role_to_searchrole(term):
    term = term.lower()
    term = re.sub("\s","",term)
    term = re.sub("[\d\-]+\.[\d\-]+\.[\d\-]+\.[\d\-]*","",term)
    term = re.sub("\#.*$","",term)
    term = re.sub("\(ec:*\)","",term)
    term = re.sub("[\(\)\[\],-]","",term)
    return term

# Read TSV file into DataFrame
gene_coords_df = pd.read_csv("data/EllenGeneCalls.tsv", sep="\t", dtype=str)

# Stash data into a dictionary based on BIOCYC gene ID
biocyc_dict = {}
for _, row in gene_coords_df.iterrows():
    geneid = None
    start = None
    stop = None
    if not pd.isna(start) and not pd.isna(stop):
        start = int(row['Start'].rstrip())
        stop = int(row['Stop'].rstrip())
    if 'BIOCYC' in row and not pd.isna(row['BIOCYC']):
        geneid = row['BIOCYC'].rstrip()
    elif 'NCBI' in row and not pd.isna(row['NCBI']):
        geneid = row['NCBI'].rstrip()
        array = row['NCBI coordinates'].split('..')
        start = int(array[0].replace(',', ''))
        array[1] = array[1].rstrip()
        stop = int(array[1].replace(',', ''))
    biocyc_dict[geneid] = row.to_dict()

genome = util.load("ADP1Genome")
gene_hash = {}
for item in genome["data"]["features"]:
    gene_hash[item["id"]] = item

anno_hash = util.load("gene_term_hash_named")

subsystems = util.load("subsystems")["response"]["docs"]
ssrole_hash = {}
for item in subsystems:
    if "role_name" not in item or not item["role_name"]:
        continue
    for role in item["role_name"]:
        srole = convert_role_to_searchrole(role)
        if srole not in ssrole_hash:
            ssrole_hash[srole] = {}
        ssrole_hash[srole][item["subsystem_name"]] = [item.get("subclass",""), item.get("class",""), item.get("superclass","")]

records = []
for geneid, data in biocyc_dict.items():
    item = {
        "ID": geneid,
        "Old ID": None,
        "Name": None,
        "Start": data.get('Start', None),
        "Stop": data.get('Stop', None),
        "Direction": None,
        "RAST": None,
        "Subsystem": None,
        "Subclass": None,
        "Class": None,
        "SuperClass": None,
    }
    if geneid in gene_hash:
        item["Direction"] = gene_hash[geneid]["location"][0][2]
        for alias in gene_hash[geneid]["aliases"]:
            if alias[0] == "gene":
                item["Name"] = alias[1]
            elif alias[0] == "old_locus_tag":
                item["Old ID"] = alias[1]
    if geneid in anno_hash:
        for attribute, value in anno_hash[geneid].items():
            if attribute[0:4] == "RAST":
                item["RAST"] = ""
                item["Subclass"] = ""
                item["Subsystem"] = ""
                item["Class"] = ""
                item["SuperClass"] = ""
                for role_id in value.keys():
                    role = value[role_id]
                    if len(item["RAST"]) > 0:
                        item["RAST"] += "; "
                    item["RAST"] += role
                    srole = convert_role_to_searchrole(role)
                    if srole in ssrole_hash:
                        for ss in ssrole_hash[srole]:
                            if len(item["Subsystem"]) > 0:
                                item["Subsystem"] += "; "
                            item["Subsystem"] += ss
                            if len(item["Subclass"]) > 0:
                                item["Subclass"] += "; "
                            item["Subclass"] += ssrole_hash[srole][ss][0]
                            if len(item["Class"]) > 0:
                                item["Class"] += "; "
                            item["Class"] += ssrole_hash[srole][ss][1]
                            if len(item["SuperClass"]) > 0:
                                item["SuperClass"] += "; "
                            item["SuperClass"] += ssrole_hash[srole][ss][2]
    records.append(item)

#Convert records to DataFrame
df = pd.DataFrame.from_records(records)
df.to_csv("data/ADP1Genes.csv", index=False)

Output files printed to:/Users/chenry/Dropbox/Contexts/Data/ADP1/nboutput when using KBDevUtils.output_dir
Loading ModelSEEDBiochem from /Users/chenry/code/ModelSEEDDatabase
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!
FOUND!


In [ ]:
 import pandas as pd

# Read TSV file into DataFrame
gene_coords_df = pd.read_csv("data/EllenGeneCalls.tsv", sep="\t", dtype=str)

# Stash data into a dictionary based on BIOCYC gene ID
biocyc_dict = {}
for _, row in gene_coords_df.iterrows():
    geneid = None
    start = None
    stop = None
    if not pd.isna(start) and not pd.isna(stop):
        start = int(row['Start'].rstrip())
        stop = int(row['Stop'].rstrip())
    if 'BIOCYC' in row and not pd.isna(row['BIOCYC']):
        geneid = row['BIOCYC'].rstrip()
    elif 'NCBI' in row and not pd.isna(row['NCBI']):
        geneid = row['NCBI'].rstrip()
        array = row['NCBI coordinates'].split('..')
        start = int(array[0].replace(',', ''))
        array[1] = array[1].rstrip()
        stop = int(array[1].replace(',', ''))
    biocyc_dict[geneid] = row.to_dict()

# Read GFF file into a list of lines
with open("data/KBase_derived_Acinetobacter_baylyi_ADP1.gff", "r") as f:
    gff_lines = f.readlines()

# Parse GFF and build a mapping from gene ID to line index
new_lines = []
found = {}
for idx, line in enumerate(gff_lines):
    if line.startswith("#") or line.strip() == "":
        new_lines.append(line)
        continue
    fields = line.strip().split("\t")
    if fields[0][0:9] == "NC_005966":
        array = fields[8].split(";")
        if array[0][3:9] == "ACIAD_":
            base_id = array[0][3:16]
            print(f"Processing {base_id}")
            found[base_id] = 1
            if base_id in biocyc_dict:
                if fields[3] != biocyc_dict[base_id]['Start']:
                    print(f"Updating {base_id} start from {fields[3]} to {biocyc_dict[base_id]['Start']}")
                if fields[4] != biocyc_dict[base_id]['Stop']:
                    print(f"Updating {base_id} stop from {fields[4]} to {biocyc_dict[base_id]['Stop']}")
                if "Notes1" in biocyc_dict[base_id] and biocyc_dict[base_id]['Notes1'] != "":
                    fields[8] += "; "+str(biocyc_dict[base_id]['Notes1'])
                if "Notes2" in biocyc_dict[base_id] and biocyc_dict[base_id]['Notes2'] != "":
                    fields[8] += "; "+str(biocyc_dict[base_id]['Notes2'])
                new_line = "\t".join(fields) + "\n"
                new_lines.append(new_line)
            else:
                new_lines.append(line)
        else:
            new_lines.append(line)
    else:
        new_lines.append(line)
for geneid in biocyc_dict:
    if geneid not in found:
        print(f"Gene {geneid} not found in GFF, adding it")
        fields = ["NC_005966.1","KBase","gene",str(biocyc_dict[geneid]['Start']),str(biocyc_dict[geneid]['Stop']),".","+","0",f"ID={geneid}"]
        new_line = "\t".join(fields) + "\n"
        new_lines.append(new_line)

with open("data/KBase_derived_Acinetobacter_baylyi_ADP1_edited.gff", "w") as f:
    f.writelines(new_lines)

In [ ]:
import pandas as pd

%run util.py

# Load the named gene-term hash
gene_term_hash_named = util.load("gene_term_hash_named")

# Prepare data for DataFrame
rows = []
for gene, sources in gene_term_hash_named.items():
    row = {"Gene": gene}
    print(sources.get("ACN2586_vs_ADP1", {}))
    row["ADP1 vs 2586"] = -1*float(sources.get("ACN2586_vs_ADP1", {}).get("value", 0))
    row["ADP1 vs 2821"] = -1*float(sources.get("ACN2821_vs_ADP1", {}).get("value", 0))
    rows.append(row)

# Create DataFrame
df = pd.DataFrame(rows)

# Print the DataFrame
print(df)

# Optionally save to a spreadsheet
df.to_excel("ADP1_comparisons.xlsx", index=False)

Output files printed to:/Users/chenry/Dropbox/Contexts/Data/ADP1/nboutput when using KBDevUtils.output_dir
Loading ModelSEEDBiochem from /Users/chenry/code/ModelSEEDDatabase
{'value': '0'}
{}
{}
{'value': '0'}
{'value': '0.901366934515238'}
{'value': '0.57766466745284'}
{}
{'value': '1.53760758584356'}
{'value': '0'}
{'value': '0'}
{'value': '2.70684063357894'}
{'value': '0'}
{}
{'value': '-1.53409577537761'}
{'value': '-1.834678397957'}
{}
{}
{}
{'value': '0'}
{}
{'value': '0'}
{'value': '0'}
{'value': '0.453876138764127'}
{'value': '0'}
{'value': '0.909729234756853'}
{'value': '0.966426329531124'}
{'value': '0'}
{'value': '0'}
{}
{'value': '0.444854180271296'}
{'value': '-0.218615932671071'}
{'value': '-1.2587898910069'}
{}
{}
{'value': '0'}
{'value': '0'}
{'value': '1.9367010816298'}
{'value': '0'}
{'value': '0'}
{}
{'value': '0'}
{'value': '3.67939133808297'}
{'value': '0.929263638872222'}
{'value': '1.90540207113952'}
{'value': '0'}
{'value': '3.20988191814523'}
{}
{'value': '0.32

In [ ]:
%run util.py
strain_data = {
    "ACN2586":"Initial construct with dgoA* and native DAHP synthesis deleted prior to amplification and evolution",
    "ACN2821":"Evolved to have a single copy of dgoA*",
    "ACN3015":"Single copy of dgoA after evolution",
    "ACN3468":"This strain has multiple copies of dgoA*",
    "ACN3471":"This strain has multiple copies of dgoA",
    "ACN3474":"Multiple copies of dgoA but partially evolved with some mutations in the genome",
    "ACN3477":"Multiple copies of dgoA but partially evolved with more mutations in the genome",
    "ADP1":"Wild type Acinetobacter baylyi ADP1"
}
util.save("strain_data", strain_data)

python version 3.9.19
KBBaseModules 0.0.1


1754023824.104131 INFO: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
1754023824.104661 INFO: NumExpr defaulting to 8 threads.


modelseedpy 0.4.2
cobrakbase 0.4.0
Output files printed to:/Users/chenry/Dropbox/Contexts/Data/ADP1/nboutput when using KBDevUtils.output_dir
Loading ModelSEEDBiochem from /Users/chenry/code/ModelSEEDDatabase


In [ ]:
# Read the ClusterSizes.txt file into a dictionary
%run util.py
cluster_sizes = {}
with open("data/ClusterSizes.txt", "r") as file:
    for line in file:
        cluster_id, size = line.strip().split("\t")
        cluster_sizes[cluster_id] = int(size)

# Save the cluster_sizes dictionary using util.save
util.save("cluster_sizes", cluster_sizes)

Output files printed to:/Users/chenry/Dropbox/Contexts/Data/ADP1/nboutput when using KBDevUtils.output_dir
Loading ModelSEEDBiochem from /Users/chenry/code/ModelSEEDDatabase


In [12]:
%run util.py
import pandas as pd

cluster_sizes = util.load("cluster_sizes")
# Load the protein cluster data
protein_cluster_df = pd.read_csv("data/Protein_HC_Cluster_Assignments.csv", dtype=str)

# Load the gene_term_hash_named data
gene_term_hash_named = util.load("gene_term_hash_named")

# Add cluster numbers to gene_term_hash_named
for _, row in protein_cluster_df.iterrows():
    gene_id = row['Protein']
    cluster_number = row['Cluster']
    if gene_id in gene_term_hash_named:
        gene_term_hash_named[gene_id]['cluster'] = cluster_number
        gene_term_hash_named[gene_id]['cluster_size'] = cluster_sizes.get(cluster_number, 0)

# Save the updated gene_term_hash_named
util.save("gene_term_hash_named", gene_term_hash_named)

Output files printed to:/Users/chenry/Dropbox/Contexts/Data/ADP1/nboutput when using KBDevUtils.output_dir
Loading ModelSEEDBiochem from /Users/chenry/code/ModelSEEDDatabase


In [7]:
%run util.py
#Adding reactions and status and flux to master gene data hash

model_gene_data = util.load("genedata")
gene_term_hash_named = util.load("gene_term_hash_named")
msflux = util.load("ms_fluxes")
msfva = util.load("ms_fva")
pubflux = util.load("pubmod_fluxes")
pubfva = util.load("pubmod_fva")

for geneid in model_gene_data:
    if geneid in gene_term_hash_named:
        gene_term_hash_named[geneid]['modelseed_flux'] = []
        gene_term_hash_named[geneid]['published_flux'] = []
        gene_term_hash_named[geneid]['modelstatus'] = model_gene_data[geneid].get('Status', 'N/A')
        gene_term_hash_named[geneid]['reactions'] = model_gene_data[geneid].get('Reactions', [])
        for rxn in model_gene_data[geneid].get('Reactions', []):
            array = rxn.split(':')
            if array[0][0:3] == "rxn":
                gene_term_hash_named[geneid]['modelseed_flux'].append(array[0]+":"+str(msflux.get(array[0], 'N/A'))+":"+str(msfva.get(array[0], ['N/A','N/A'])))
                gene_term_hash_named[geneid]['published_flux'].append("N/A")
            else:
                idarray = array[0].split("(")
                pubid = idarray[0]
                if len(idarray) > 1:
                    msid = idarray[1][0:-1]
                    if msid in msflux:
                        gene_term_hash_named[geneid]['modelseed_flux'].append(msid+":"+str(msflux.get(msid, 'N/A'))+":"+str(msfva.get(msid, ['N/A','N/A'])))
                    else:
                        gene_term_hash_named[geneid]['modelseed_flux'].append("N/A")
                else:
                    gene_term_hash_named[geneid]['modelseed_flux'].append("N/A")
                gene_term_hash_named[geneid]['published_flux'].append(pubid+":"+str(pubflux.get(pubid, 'N/A'))+":"+str(pubfva.get(pubid, ['N/A','N/A'])))
util.save("gene_term_hash_named", gene_term_hash_named)

2025-09-26 09:51:13,343 - __main__.NotebookUtil - INFO - Loaded 0 tokens from /Users/chenry/.tokens
2025-09-26 09:51:13,344 - __main__.NotebookUtil - INFO - Loaded kbase tokens from /Users/chenry/.kbase/token
2025-09-26 09:51:13,345 - __main__.NotebookUtil - WARNING - Section 'ModelSEEDBiochem' not found in config
2025-09-26 09:51:13,345 - __main__.NotebookUtil - INFO - ModelSEED database loaded from /Users/chenry/Dropbox/SyncCode/KBUtilLib/src/kbutillib/dependencies/ModelSEEDDatabase


/Users/chenry/Dropbox/SyncCode/KBUtilLib/src


2025-09-26 09:51:13,738 - __main__.NotebookUtil - INFO - Notebook environment detected
2025-09-26 09:51:13,738 - __main__.NotebookUtil - INFO - ArgoGatewayClient initialised | model=gpto3mini env=dev timeout=120.0s url=https://apps-dev.inside.anl.gov/argoapi/api/v1/resource/streamchat/


In [13]:
#Adding raw proteome values to gene data
%run util.py
gene_term_hash_named = util.load("gene_term_hash_named")
# Load the worksheet named "Imputed" into a DataFrame
df = pd.read_excel("data/UGA_Proteomics_May2025_Report.xlsx", sheet_name="Imputed", header=1)

# Columns that you want to average
cols_to_avg = [f"ACN2586_{i}" for i in range(1, 6)]
other_cols_to_avg = [f"ACN2821_{i}" for i in range(1, 6)]

# Iterate through each row and print Protein Accession + average
for _, row in df.iterrows():
    protein = row["Protein Accession"]
    avg_val = row[cols_to_avg].mean()
    avg_val_two = row[other_cols_to_avg].mean()
    if protein in gene_term_hash_named:
        gene_term_hash_named[protein]['ACN2586_proteome'] = avg_val
        gene_term_hash_named[protein]['ACN2821_proteome'] = avg_val_two

util.save("gene_term_hash_named", gene_term_hash_named)

2025-09-26 10:26:30,340 - __main__.NotebookUtil - INFO - Loaded 0 tokens from /Users/chenry/.tokens
2025-09-26 10:26:30,341 - __main__.NotebookUtil - INFO - Loaded kbase tokens from /Users/chenry/.kbase/token
2025-09-26 10:26:30,341 - __main__.NotebookUtil - WARNING - Section 'ModelSEEDBiochem' not found in config
2025-09-26 10:26:30,342 - __main__.NotebookUtil - INFO - ModelSEED database loaded from /Users/chenry/Dropbox/SyncCode/KBUtilLib/src/kbutillib/dependencies/ModelSEEDDatabase


/Users/chenry/Dropbox/SyncCode/KBUtilLib/src


2025-09-26 10:26:30,717 - __main__.NotebookUtil - INFO - Notebook environment detected
2025-09-26 10:26:30,717 - __main__.NotebookUtil - INFO - ArgoGatewayClient initialised | model=gpto3mini env=dev timeout=120.0s url=https://apps-dev.inside.anl.gov/argoapi/api/v1/resource/streamchat/
/opt/anaconda3/envs/modelseed/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


# Creating comprehensive gene dataframe with sequences and reactions

In [1]:
%run util.py
import pandas as pd
import json

# Load the gene list
with open("data/ADP1GeneList.txt", "r") as f:
    gene_list = [line.strip() for line in f if line.strip()]

# Load genome data
genome_data = util.load("ADP1Genome")

# Load published model
with open("data/FullyTranslatedPublishedModel.json", "r") as f:
    model_data = json.load(f)

# Load annotations
gene_term_hash_named = util.load("gene_term_hash_named")

# Create mappings from genome data
gene_to_cds = {}
for cds in genome_data["data"]["cdss"]:
    parent_gene = cds.get("parent_gene")
    if parent_gene:
        gene_to_cds[parent_gene] = cds

# Create gene to reactions mapping from model
gene_to_reactions = {}
metabolite_names = {met["id"]: met["name"] for met in model_data["metabolites"]}

for reaction in model_data["reactions"]:
    rxn_id = reaction["id"]
    
    # Build reaction equation with metabolite names
    reactants = []
    products = []
    for met_id, coeff in reaction["metabolites"].items():
        met_name = metabolite_names.get(met_id, met_id)
        if coeff < 0:
            if abs(coeff) == 1:
                reactants.append(met_name)
            else:
                reactants.append(f"{abs(coeff):g} {met_name}")
        elif coeff > 0:
            if coeff == 1:
                products.append(met_name)
            else:
                products.append(f"{coeff:g} {met_name}")
    
    equation = " + ".join(reactants) + " <=> " + " + ".join(products)
    rxn_string = f"{rxn_id}: {equation}"
    
    # Parse gene reaction rule
    grr = reaction.get("gene_reaction_rule", "")
    if grr:
        # Simple parsing - split by 'or' and 'and', clean up
        genes = grr.replace("(", "").replace(")", "").replace(" and ", " ").replace(" or ", " ").split()
        for gene in genes:
            gene = gene.strip()
            if gene:
                if gene not in gene_to_reactions:
                    gene_to_reactions[gene] = []
                gene_to_reactions[gene].append(rxn_string)

# Build dataframe
rows = []
for gene_id in gene_list:
    row = {"Gene ID": gene_id}
    
    # Add alternative annotations from gene_term_hash_named
    if gene_id in gene_term_hash_named:
        annotations = gene_term_hash_named[gene_id]
        
        # Collect RAST annotations
        rast_key = "RAST-annotate_genome:1.9.1:SSO:2025-06-06T03:38:24"
        if rast_key in annotations and isinstance(annotations[rast_key], dict):
            row["RAST"] = "; ".join(str(v) for v in annotations[rast_key].values())
        
        # Collect other annotations and merge them
        other_annos = []
        
        # GLM4EC
        glm4ec_key = "GLM4ECModule.annotate_microbes_with_GLM4EC:0.1.1.am:EC:2025-06-06 03:51:01"
        if glm4ec_key in annotations and isinstance(annotations[glm4ec_key], dict):
            for anno in annotations[glm4ec_key].values():
                other_annos.append(f"GLM4EC: {anno}")
        
        # DRAM
        dram_key = "DRAM:0.1.2:KO:2025_06_06_04_17_25"
        if dram_key in annotations and isinstance(annotations[dram_key], dict):
            for anno in annotations[dram_key].values():
                other_annos.append(f"DRAM: {anno}")
        
        # Snekmer
        snekmer_key = "Snekmer Apply:1.0:PF:2025.06.06-04:57:01AM"
        if snekmer_key in annotations and isinstance(annotations[snekmer_key], dict):
            for anno in annotations[snekmer_key].values():
                other_annos.append(f"Snekmer: {anno}")
        
        row["Other annotations"] = " | ".join(other_annos) if other_annos else ""
        
        # Add Uniprot ID and PLM score
        row["Uniprot ID"] = annotations.get("uniprot_id", "")
        row["Uniprot PLM Score"] = annotations.get("uniprot_plm_score", "")
    else:
        row["Uniprot ID"] = ""
        row["Uniprot PLM Score"] = ""
    
    # Add reactions from model
    if gene_id in gene_to_reactions:
        row["Reactions"] = " | ".join(gene_to_reactions[gene_id])
    else:
        row["Reactions"] = ""
    
    # Add DNA and protein sequences from genome
    if gene_id in gene_to_cds:
        cds = gene_to_cds[gene_id]
        row["DNA Sequence"] = cds.get("dna_sequence", "")
        row["Protein Sequence"] = cds.get("protein_translation", "")
    else:
        row["DNA Sequence"] = ""
        row["Protein Sequence"] = ""
    
    rows.append(row)

# Create DataFrame
df = pd.DataFrame(rows)

# Print to TSV
output_path = "nboutput/gene_comprehensive_data.tsv"
df.to_csv(output_path, sep="\t", index=False)
print(f"Saved comprehensive gene data to {output_path}")
print(f"Total genes: {len(df)}")
print(f"Columns: {', '.join(df.columns)}")
print(f"\nFirst few rows:")
print(df.head())

/Users/chenry/Dropbox/Projects/KBUtilLib/src


2025-11-12 09:37:38,996 - __main__.NotebookUtil - INFO - Loaded 0 tokens from /Users/chenry/.tokens
2025-11-12 09:37:38,997 - __main__.NotebookUtil - INFO - Loaded kbase tokens from /Users/chenry/.kbase/token
2025-11-12 09:37:38,998 - __main__.NotebookUtil - WARNING - Section 'ModelSEEDBiochem' not found in config


modelseedpy 0.4.2
cobrakbase 0.4.0


2025-11-12 09:37:43,060 - __main__.NotebookUtil - INFO - ModelSEED database loaded from /Users/chenry/Dropbox/Projects/KBUtilLib/src/kbutillib/dependencies/ModelSEEDDatabase
2025-11-12 09:37:43,415 - __main__.NotebookUtil - WARNING - BLAST tools not found. Install NCBI BLAST+ to use BLAST functionality. On Ubuntu/Debian: sudo apt-get install ncbi-blast+, On MacOS: brew install blast
2025-11-12 09:37:43,416 - __main__.NotebookUtil - INFO - Notebook environment detected
2025-11-12 09:37:43,416 - __main__.NotebookUtil - INFO - ArgoGatewayClient initialised | model=gpto3mini env=dev timeout=120.0s url=https://apps-dev.inside.anl.gov/argoapi/api/v1/resource/streamchat/


Saved comprehensive gene data to nboutput/gene_comprehensive_data.tsv
Total genes: 3356
Columns: Gene ID, RAST, Other annotations, Uniprot ID, Uniprot PLM Score, Reactions, DNA Sequence, Protein Sequence

First few rows:
         Gene ID                                            RAST  \
0  ACIAD_RS00005  Chromosomal replication initiator protein DnaA   
1  ACIAD_RS00010    DNA polymerase III beta subunit (EC 2.7.7.7)   
2  ACIAD_RS00015       DNA recombination and repair protein RecF   
3  ACIAD_RS00020              DNA gyrase subunit B (EC 5.99.1.3)   
4  ACIAD_RS00025                                             NaN   

                                   Other annotations Uniprot ID  \
0  DRAM: chromosomal replication initiator protei...     B2HZA7   
1  GLM4EC: DNA-directed DNA polymerase. | DRAM: D...     P43744   
2  DRAM: DNA replication and repair protein RecF ...     A3M0Q6   
3  GLM4EC: 1 | DRAM: DNA gyrase subunit B [EC:5.9...     P0AES7   
4  Snekmer:  Uncharacterized protei

# Merge pairwise strain comparison proteomics data with comprehensive gene data

In [2]:
import pandas as pd

# Load the Tukey proteomics data with pairwise strain comparisons
tukey_df = pd.read_excel("data/UGA_Proteomics_May2025_Tukeys.xlsx", header=1)

# Create a dictionary to store all pairwise comparisons for each gene
# Key: gene_id, Value: dict of comparison columns (just fold changes)
gene_comparisons = {}

for _, row in tukey_df.iterrows():
    gene_id = row['variable']
    group1 = row['group1']
    group2 = row['group2']
    estimate = row['estimate']  # Log2 Fold Change (group2 - group1)
    
    # Create comparison column name (just fold change)
    comparison_name = f"{group1}_vs_{group2}"
    
    # Initialize gene entry if not exists
    if gene_id not in gene_comparisons:
        gene_comparisons[gene_id] = {}
    
    # Store the comparison data
    gene_comparisons[gene_id][comparison_name] = estimate

# Convert the nested dictionary to a dataframe
comparisons_df = pd.DataFrame.from_dict(gene_comparisons, orient='index')
comparisons_df.reset_index(inplace=True)
comparisons_df.rename(columns={'index': 'Protein ID'}, inplace=True)

# Save the comparison dataframe (just protein IDs and fold changes)
output_path = "nboutput/gene_pairwise_comparisons.tsv"
comparisons_df.to_csv(output_path, sep="\t", index=False)

print(f"Saved pairwise comparison data to {output_path}")
print(f"Total genes: {len(comparisons_df)}")
print(f"Comparison columns: {len(comparisons_df.columns) - 1}")  # -1 for Protein ID
print(f"\nComparison columns:")
comparison_cols = [col for col in comparisons_df.columns if col != 'Protein ID']
for col in sorted(comparison_cols):
    print(f"  - {col}")
print(f"\nFirst few rows:")
print(comparisons_df.head())

Saved pairwise comparison data to nboutput/gene_pairwise_comparisons.tsv
Total genes: 2158
Comparison columns: 28

Comparison columns:
  - ACN2586_vs_ACN2821
  - ACN2586_vs_ACN3015
  - ACN2586_vs_ACN3468
  - ACN2586_vs_ACN3471
  - ACN2586_vs_ACN3474
  - ACN2586_vs_ACN3477
  - ACN2586_vs_ADP1
  - ACN2821_vs_ACN3015
  - ACN2821_vs_ACN3468
  - ACN2821_vs_ACN3471
  - ACN2821_vs_ACN3474
  - ACN2821_vs_ACN3477
  - ACN2821_vs_ADP1
  - ACN3015_vs_ACN3468
  - ACN3015_vs_ACN3471
  - ACN3015_vs_ACN3474
  - ACN3015_vs_ACN3477
  - ACN3015_vs_ADP1
  - ACN3468_vs_ACN3471
  - ACN3468_vs_ACN3474
  - ACN3468_vs_ACN3477
  - ACN3468_vs_ADP1
  - ACN3471_vs_ACN3474
  - ACN3471_vs_ACN3477
  - ACN3471_vs_ADP1
  - ACN3474_vs_ACN3477
  - ACN3474_vs_ADP1
  - ACN3477_vs_ADP1

First few rows:
      Protein ID  ACN2586_vs_ACN2821  ACN2586_vs_ACN3015  ACN2586_vs_ACN3468  \
0           DgoA           -0.519406           -9.287057            3.315047   
1        DgoA_Ec            0.591271           -0.220321         

# Finding closest Uniprot IDs using PLM API

In [ ]:
%run util.py

# Load the gene list
with open("data/ADP1GeneList.txt", "r") as f:
    gene_list = [line.strip() for line in f if line.strip()]

# Load genome data to get protein sequences
genome_data = util.load("ADP1Genome")

# Load existing gene_term_hash_named
gene_term_hash_named = util.load("gene_term_hash_named")

# Build mapping from gene ID to CDS for protein sequences
gene_to_cds = {}
for cds in genome_data["data"]["cdss"]:
    parent_gene = cds.get("parent_gene")
    if parent_gene:
        gene_to_cds[parent_gene] = cds

# Prepare query sequences for PLM API
query_sequences = []
genes_already_have_uniprot = 0

for gene_id in gene_list:
    # Ensure gene exists in gene_term_hash_named
    if gene_id not in gene_term_hash_named:
        gene_term_hash_named[gene_id] = {}
    
    # Skip if we already have a Uniprot ID
    if "uniprot_id" in gene_term_hash_named[gene_id]:
        genes_already_have_uniprot += 1
        continue
    
    # Get protein sequence
    if gene_id in gene_to_cds:
        protein_seq = gene_to_cds[gene_id].get("protein_translation", "")
        if protein_seq:
            query_sequences.append({
                "id": gene_id,
                "sequence": protein_seq
            })

print(f"Genes to query: {len(query_sequences)} (skipping {genes_already_have_uniprot} already with Uniprot IDs)")

if len(query_sequences) == 0:
    print("No genes to query! All genes either have Uniprot IDs or lack sequences.")
else:
    # Query PLM API in batches
    batch_size = 50
    total_batches = (len(query_sequences) + batch_size - 1) // batch_size
    
    total_genes_with_hits = 0
    total_hits_count = 0
    
    print(f"Processing {total_batches} batches...")
    
    for batch_idx in range(total_batches):
        start_idx = batch_idx * batch_size
        end_idx = min(start_idx + batch_size, len(query_sequences))
        batch = query_sequences[start_idx:end_idx]
        
        print(f"  Batch {batch_idx + 1}/{total_batches}: ", end="", flush=True)
        
        try:
            # Query PLM API for homologs
            plm_results = util.query_plm_api(
                batch,
                max_hits=5,
                similarity_threshold=0.0,
                return_embeddings=False
            )
            
            # Process results
            hits_list = plm_results.get("hits", [])
            batch_genes_with_hits = 0
            
            for hits_data in hits_list:
                query_id = hits_data.get("query_id", "")
                hits = hits_data.get("hits", [])
                
                if hits:
                    batch_genes_with_hits += 1
                    total_hits_count += len(hits)
                    
                    # Get the best hit (first one, highest score)
                    best_hit = hits[0]
                    uniprot_id = best_hit.get("id", "")
                    score = best_hit.get("score", 0.0)
                    
                    # Store in gene_term_hash_named
                    gene_term_hash_named[query_id]["uniprot_id"] = uniprot_id
                    gene_term_hash_named[query_id]["uniprot_plm_score"] = score
                else:
                    gene_term_hash_named[query_id]["uniprot_id"] = None
                    gene_term_hash_named[query_id]["uniprot_plm_score"] = None
            
            total_genes_with_hits += batch_genes_with_hits
            print(f"{batch_genes_with_hits}/{len(batch)} genes with hits")
        
        except Exception as e:
            print(f"ERROR: {str(e)}")
            import traceback
            traceback.print_exc()
    
    # Save updated gene_term_hash_named
    util.save("gene_term_hash_named", gene_term_hash_named)
    
    # Print final summary
    genes_with_uniprot = sum(1 for g in gene_list if g in gene_term_hash_named and gene_term_hash_named[g].get("uniprot_id"))
    
    print(f"\nCompleted! Results:")
    print(f"  Total genes: {len(gene_list)}")
    print(f"  Genes with Uniprot IDs: {genes_with_uniprot}")
    print(f"  Newly added: {total_genes_with_hits}")
    print(f"  Average hits per gene: {total_hits_count / total_genes_with_hits if total_genes_with_hits > 0 else 0:.1f}")